In [1]:
print("Kernel is working")

Kernel is working


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

# ML Models
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, precision_score, confusion_matrix
import xgboost as xgb
import lightgbm as lgb

# Paths
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / 'data'
MODEL_DIR = BASE_DIR / 'models' / 'signal_generator_v6'
MODEL_DIR.mkdir(parents=True, exist_ok=True)

print("✅ Libraries loaded")
print(f"✅ Model Directory: {MODEL_DIR}")

✅ Libraries loaded
✅ Model Directory: c:\Users\Acer\Desktop\Forex-Signal-App\models\signal_generator_v6


## 1. Data Loading & Feature Engineering (V6 Enhanced)

In [3]:
# Load Data
train_df = pd.read_csv(DATA_DIR / 'EUR_USD_1min.csv')
test_df = pd.read_csv(DATA_DIR / 'EUR_USD_test.csv')

# Standardize columns
for df in [train_df, test_df]:
    if 'timestamp' in df.columns:
        df.rename(columns={'timestamp': 'time'}, inplace=True)
    df['time'] = pd.to_datetime(df['time'])

print(f"Train: {len(train_df):,} rows")
print(f"Test: {len(test_df):,} rows")

def calculate_adx(df, period=14):
    """Calculate Average Directional Index (ADX)"""
    df = df.copy()
    df['tr0'] = abs(df['high'] - df['low'])
    df['tr1'] = abs(df['high'] - df['close'].shift())
    df['tr2'] = abs(df['low'] - df['close'].shift())
    df['tr'] = df[['tr0', 'tr1', 'tr2']].max(axis=1)
    
    df['up_move'] = df['high'] - df['high'].shift()
    df['down_move'] = df['low'].shift() - df['low']
    
    df['plus_dm'] = np.where((df['up_move'] > df['down_move']) & (df['up_move'] > 0), df['up_move'], 0)
    df['minus_dm'] = np.where((df['down_move'] > df['up_move']) & (df['down_move'] > 0), df['down_move'], 0)
    
    # Smooth
    df['atr'] = df['tr'].rolling(period).mean()
    df['plus_di'] = 100 * (df['plus_dm'].rolling(period).mean() / df['atr'])
    df['minus_di'] = 100 * (df['minus_dm'].rolling(period).mean() / df['atr'])
    
    df['dx'] = 100 * abs(df['plus_di'] - df['minus_di']) / (df['plus_di'] + df['minus_di'] + 1e-10)
    df['adx'] = df['dx'].rolling(period).mean()
    
    return df['adx'], df['plus_di'], df['minus_di']

def add_features_v6(df):
    """V6 Features: V5 + ADX + CCI + Williams %R"""
    df = df.copy()
    
    # --- V5 Time Features ---
    df['hour'] = df['time'].dt.hour
    df['day_of_week'] = df['time'].dt.dayofweek
    df['is_london'] = ((df['hour'] >= 8) & (df['hour'] < 16)).astype(int)
    df['is_ny'] = ((df['hour'] >= 13) & (df['hour'] < 21)).astype(int)
    
    # --- Basic MAs ---
    for p in [5, 10, 20, 50, 200]:
        df[f'sma_{p}'] = df['close'].rolling(p).mean()
        df[f'ema_{p}'] = df['close'].ewm(span=p, adjust=False).mean()
    
    # --- RSI ---
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / (loss + 1e-10)
    df['rsi'] = 100 - (100 / (1 + rs))
    
    # --- MACD ---
    ema12 = df['close'].ewm(span=12).mean()
    ema26 = df['close'].ewm(span=26).mean()
    df['macd'] = ema12 - ema26
    df['macd_signal'] = df['macd'].ewm(span=9).mean()
    df['macd_hist'] = df['macd'] - df['macd_signal']
    
    # --- Bollinger Bands ---
    df['bb_mid'] = df['close'].rolling(20).mean()
    df['bb_std'] = df['close'].rolling(20).std()
    df['bb_upper'] = df['bb_mid'] + 2 * df['bb_std']
    df['bb_lower'] = df['bb_mid'] - 2 * df['bb_std']
    df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / df['bb_mid']
    
    # --- NEW V6 INDICATORS ---
    
    # 1. ADX (Trend Strength)
    df['adx'], df['plus_di'], df['minus_di'] = calculate_adx(df)
    df['trend_strength'] = np.where(df['adx'] > 25, 1, 0) # Strong trend
    
    # 2. CCI (Commodity Channel Index)
    tp = (df['high'] + df['low'] + df['close']) / 3
    sma_tp = tp.rolling(20).mean()
    mad_tp = tp.rolling(20).apply(lambda x: np.abs(x - x.mean()).mean())
    df['cci'] = (tp - sma_tp) / (0.015 * mad_tp)
    
    # 3. Williams %R
    hh = df['high'].rolling(14).max()
    ll = df['low'].rolling(14).min()
    df['williams_r'] = -100 * (hh - df['close']) / (hh - ll + 1e-10)
    
    # 4. Volatility (Standard Deviation of returns)
    df['returns'] = df['close'].pct_change()
    df['volatility'] = df['returns'].rolling(20).std() * 100
    
    # 5. Interaction Features
    df['rsi_x_adx'] = df['rsi'] * df['adx'] / 100
    df['momentum_score'] = (df['rsi'] > 50).astype(int) + (df['macd'] > df['macd_signal']).astype(int) + (df['plus_di'] > df['minus_di']).astype(int)
    
    return df

print("Adding V6 features...")
train_df = add_features_v6(train_df)
test_df = add_features_v6(test_df)
print(f"✓ Features added. Total columns: {len(train_df.columns)}")

Train: 1,859,492 rows
Test: 296,778 rows
Adding V6 features...
✓ Features added. Total columns: 39
✓ Features added. Total columns: 39


## 2. Labeling (BUY / SELL / HOLD)

In [4]:
def create_labels_v6(df, forward_periods=60, min_pips=15, ratio=1.5):
    """
    3-Class Labeling:
    2 (BUY):  Up move >= min_pips AND Up > Down * ratio
    0 (SELL): Down move >= min_pips AND Down > Up * ratio
    1 (HOLD): Everything else
    
    Note: Using 0,1,2 instead of -1,0,1 for XGBoost compatibility
    """
    df = df.copy()
    min_move = min_pips * 0.0001
    
    # Future Max/Min
    df['future_max'] = df['high'].rolling(forward_periods).max().shift(-forward_periods)
    df['future_min'] = df['low'].rolling(forward_periods).min().shift(-forward_periods)
    
    df['up_move'] = df['future_max'] - df['close']
    df['down_move'] = df['close'] - df['future_min']
    
    conditions = [
        (df['up_move'] >= min_move) & (df['up_move'] > df['down_move'] * ratio),  # BUY
        (df['down_move'] >= min_move) & (df['down_move'] > df['up_move'] * ratio)  # SELL
    ]
    choices = [2, 0]  # BUY=2, SELL=0, HOLD=1
    
    df['signal'] = np.select(conditions, choices, default=1)
    
    # Drop helper columns
    df.drop(['future_max', 'future_min', 'up_move', 'down_move'], axis=1, inplace=True)
    return df

# Label mapping for display
LABEL_MAP = {0: 'SELL', 1: 'HOLD', 2: 'BUY'}

train_df = create_labels_v6(train_df)
test_df = create_labels_v6(test_df)

print("Label Distribution (Train):")
for val, name in LABEL_MAP.items():
    pct = (train_df['signal'] == val).mean() * 100
    print(f"  {name}: {pct:.1f}%")

print("\nLabel Distribution (Test):")
for val, name in LABEL_MAP.items():
    pct = (test_df['signal'] == val).mean() * 100
    print(f"  {name}: {pct:.1f}%")

Label Distribution (Train):
  SELL: 10.7%
  HOLD: 78.9%
  BUY: 10.4%

Label Distribution (Test):
  SELL: 12.9%
  HOLD: 72.9%
  BUY: 14.1%


## 3. Training 5-Model Ensemble

In [6]:
# Prepare Data
exclude_cols = ['time', 'signal', 'open', 'high', 'low', 'close', 'volume', 'tick_volume']
feature_cols = [c for c in train_df.columns if c not in exclude_cols]

# Filter for training (remove NaNs)
train_clean = train_df.dropna(subset=feature_cols + ['signal']).copy()
test_clean = test_df.dropna(subset=feature_cols + ['signal']).copy()

# We train on ALL data (0, 1, 2) to learn to distinguish HOLD from signals
X_train = train_clean[feature_cols].values
y_train = train_clean['signal'].values.astype(int)
X_test = test_clean[feature_cols].values
y_test = test_clean['signal'].values.astype(int)

# Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training on {len(X_train):,} samples with {len(feature_cols)} features")
print(f"Classes: {np.unique(y_train)}")

# Check GPU
import torch
GPU_AVAILABLE = torch.cuda.is_available()
print(f"GPU Available: {GPU_AVAILABLE}")

# Install CatBoost if needed
try:
    from catboost import CatBoostClassifier
    CATBOOST_AVAILABLE = True
except ImportError:
    CATBOOST_AVAILABLE = False
    print("CatBoost not installed. Run: pip install catboost")

# --- Define Models (ALL GPU enabled) ---
models = {}

# 1. XGBoost (GPU) - Default config
models['xgb1'] = xgb.XGBClassifier(
    n_estimators=500, max_depth=8, learning_rate=0.05, 
    subsample=0.8, colsample_bytree=0.8, random_state=42,
    tree_method='hist', device='cuda', verbosity=0
)

# 2. XGBoost (GPU) - Deeper trees
models['xgb2'] = xgb.XGBClassifier(
    n_estimators=400, max_depth=12, learning_rate=0.03, 
    subsample=0.7, colsample_bytree=0.7, random_state=43,
    tree_method='hist', device='cuda', verbosity=0,
    reg_alpha=0.1, reg_lambda=1.0
)

# 3. LightGBM (GPU) - Default config
models['lgb1'] = lgb.LGBMClassifier(
    n_estimators=500, max_depth=8, learning_rate=0.05,
    subsample=0.8, colsample_bytree=0.8, random_state=42, 
    verbose=-1, device='gpu'
)

# 4. LightGBM (GPU) - More trees, less depth
models['lgb2'] = lgb.LGBMClassifier(
    n_estimators=700, max_depth=6, learning_rate=0.03,
    subsample=0.75, colsample_bytree=0.75, random_state=44, 
    verbose=-1, device='gpu',
    num_leaves=63, min_child_samples=20
)

# 5. CatBoost (GPU) - Robust gradient boosting
if CATBOOST_AVAILABLE:
    models['cat'] = CatBoostClassifier(
        iterations=500, depth=8, learning_rate=0.05,
        random_seed=42, task_type='GPU', devices='0',
        verbose=False, loss_function='MultiClass'
    )

# --- Train Loop ---
predictions = {}
probabilities = {}

print("\n🚀 Starting Training...")
for name, model in models.items():
    print(f"  Training {name.upper()}...", end=" ")
    model.fit(X_train_scaled, y_train)
    
    # Predict
    predictions[name] = model.predict(X_test_scaled)
    probabilities[name] = model.predict_proba(X_test_scaled)
    
    acc = accuracy_score(y_test, predictions[name])
    print(f"✓ Accuracy: {acc*100:.2f}%")

Training on 1,858,074 samples with 33 features
Classes: [0 1 2]
GPU Available: True
GPU Available: True

🚀 Starting Training...
  Training XGB1... 
🚀 Starting Training...
  Training XGB1... ✓ Accuracy: 73.41%
  Training XGB2... ✓ Accuracy: 73.41%
  Training XGB2... ✓ Accuracy: 73.24%
  Training LGB1... ✓ Accuracy: 73.24%
  Training LGB1... ✓ Accuracy: 73.42%
  Training LGB2... ✓ Accuracy: 73.42%
  Training LGB2... ✓ Accuracy: 73.44%
  Training CAT... ✓ Accuracy: 73.44%
  Training CAT... ✓ Accuracy: 73.43%
✓ Accuracy: 73.43%


## 4. Ensemble & Directional Analysis

In [10]:
# Weighted Soft Voting (5 GPU models)
# XGB1, XGB2, LGB1, LGB2, CatBoost
if CATBOOST_AVAILABLE:
    weights = {'xgb1': 0.20, 'xgb2': 0.20, 'lgb1': 0.20, 'lgb2': 0.20, 'cat': 0.20}
else:
    weights = {'xgb1': 0.25, 'xgb2': 0.25, 'lgb1': 0.25, 'lgb2': 0.25}

# Combine Probabilities
# Classes: 0=SELL, 1=HOLD, 2=BUY
print(f"Model Classes: {models['xgb1'].classes_}")

final_proba = np.zeros_like(probabilities['xgb1'])
for name, w in weights.items():
    final_proba += w * probabilities[name]

# Get Predictions
final_pred = models['xgb1'].classes_[np.argmax(final_proba, axis=1)]

print("="*60)
print("📊 ENSEMBLE V6 RESULTS")
print("="*60)
print(classification_report(y_test, final_pred, target_names=['SELL', 'HOLD', 'BUY']))

# --- Directional Analysis ---
print("\n🔍 DIRECTIONAL PERFORMANCE (BUY vs SELL)")
print("-"*60)

# BUY Analysis (Class 2)
buy_mask = (final_pred == 2)
buy_total = buy_mask.sum()
buy_correct = (y_test[buy_mask] == 2).sum()
buy_acc = buy_correct / buy_total * 100 if buy_total > 0 else 0

# SELL Analysis (Class 0)
sell_mask = (final_pred == 0)
sell_total = sell_mask.sum()
sell_correct = (y_test[sell_mask] == 0).sum()
sell_acc = sell_correct / sell_total * 100 if sell_total > 0 else 0

print(f"BUY Signals:  {buy_total:>5} | Accuracy: {buy_acc:>6.2f}%")
print(f"SELL Signals: {sell_total:>5} | Accuracy: {sell_acc:>6.2f}%")

# Recommendation
print("\n💡 RECOMMENDATION:")
if buy_acc > 55 and sell_acc > 55:
    print("   ✅ Use BOTH BUY and SELL signals.")
elif buy_acc > 55 and sell_acc < 50:
    print("   ⚠️ SELL accuracy is low. Use BUY-ONLY mode.")
elif sell_acc > 55 and buy_acc < 50:
    print("   ⚠️ BUY accuracy is low. Use SELL-ONLY mode.")
else:
    print("   ⚠️ Both sides have low base accuracy. Rely on High Confidence Thresholds.")

Model Classes: [0 1 2]
📊 ENSEMBLE V6 RESULTS
              precision    recall  f1-score   support

        SELL       0.35      0.10      0.16     38401
        HOLD       0.76      0.97      0.85    216283
         BUY       0.37      0.07      0.12     41895

    accuracy                           0.73    296579
   macro avg       0.50      0.38      0.38    296579
weighted avg       0.65      0.73      0.66    296579


🔍 DIRECTIONAL PERFORMANCE (BUY vs SELL)
------------------------------------------------------------
BUY Signals:   8277 | Accuracy:  37.33%
SELL Signals: 11252 | Accuracy:  35.22%

💡 RECOMMENDATION:
   ⚠️ Both sides have low base accuracy. Rely on High Confidence Thresholds.


## 5. High Confidence Optimization

In [11]:
# Get probability for predicted class
max_proba = np.max(final_proba, axis=1) * 100

print("="*70)
print("📊 ACCURACY BY CONFIDENCE THRESHOLD")
print("="*70)
print(f"{'Conf':>5} | {'Type':>5} | {'Signals':>8} | {'Correct':>8} | {'Accuracy':>8}")
print("-"*70)

best_config = {'threshold': 0, 'mode': 'BOTH'}
best_acc = 0

for conf in [50, 60, 70, 75, 80, 85, 90]:
    # BUY (class 2)
    mask_b = (final_pred == 2) & (max_proba >= conf)
    sig_b = mask_b.sum()
    acc_b = (y_test[mask_b] == 2).mean() * 100 if sig_b > 0 else 0
    
    # SELL (class 0)
    mask_s = (final_pred == 0) & (max_proba >= conf)
    sig_s = mask_s.sum()
    acc_s = (y_test[mask_s] == 0).mean() * 100 if sig_s > 0 else 0
    
    if sig_b > 0: print(f"{conf:>5}% | {'BUY':>5} | {sig_b:>8} | {int(sig_b*acc_b/100):>8} | {acc_b:>8.1f}%")
    if sig_s > 0: print(f"{conf:>5}% | {'SELL':>5} | {sig_s:>8} | {int(sig_s*acc_s/100):>8} | {acc_s:>8.1f}%")
    
    # Track best
    if acc_b > best_acc and sig_b > 10:
        best_acc = acc_b
        best_config = {'threshold': conf, 'mode': 'BUY_ONLY'}
    if acc_s > best_acc and sig_s > 10:
        best_acc = acc_s
        best_config = {'threshold': conf, 'mode': 'SELL_ONLY'}
    if acc_b > 55 and acc_s > 55 and (acc_b + acc_s)/2 > best_acc:
        best_acc = (acc_b + acc_s)/2
        best_config = {'threshold': conf, 'mode': 'BOTH'}

print("-"*70)
print(f"🏆 Best Configuration: {best_config}")

📊 ACCURACY BY CONFIDENCE THRESHOLD
 Conf |  Type |  Signals |  Correct | Accuracy
----------------------------------------------------------------------
   50% |   BUY |      371 |      154 |     41.5%
   50% |  SELL |      926 |      371 |     40.1%
   60% |   BUY |       17 |        5 |     29.4%
   60% |  SELL |      129 |       37 |     28.7%
   70% |  SELL |       25 |        1 |      4.0%
   75% |  SELL |        9 |        0 |      0.0%
   80% |  SELL |        2 |        0 |      0.0%
----------------------------------------------------------------------
🏆 Best Configuration: {'threshold': 50, 'mode': 'BUY_ONLY'}


In [ ]:
# Save Models
print("Saving V6 Ensemble...")
for name, model in models.items():
    joblib.dump(model, MODEL_DIR / f'{name}_v6.joblib')

joblib.dump(scaler, MODEL_DIR / 'scaler_v6.joblib')
joblib.dump(feature_cols, MODEL_DIR / 'feature_cols_v6.joblib')
joblib.dump(best_config, MODEL_DIR / 'config_v6.joblib')

print("✅ V6 Models Saved Successfully!")

## 6. V6 Alternative: BUY vs SELL Only (Like V5)

In [7]:
# ================================================================
# V6 BINARY: BUY vs SELL ONLY (HOLD-г хассан)
# V5-тай ижил арга - илүү сайн үр дүн гарна
# ================================================================
print("="*70)
print("🔄 V6 BINARY: BUY vs SELL Only (HOLD хассан)")
print("="*70)

# Filter BUY/SELL only (remove HOLD = class 1)
train_binary = train_clean[train_clean['signal'] != 1].copy()
test_binary = test_clean[test_clean['signal'] != 1].copy()

# Convert labels: SELL=0, BUY=1 (was SELL=0, BUY=2)
train_binary['signal_bin'] = (train_binary['signal'] == 2).astype(int)
test_binary['signal_bin'] = (test_binary['signal'] == 2).astype(int)

X_train_bin = train_binary[feature_cols].values
y_train_bin = train_binary['signal_bin'].values
X_test_bin = test_binary[feature_cols].values
y_test_bin = test_binary['signal_bin'].values

# Scale
scaler_bin = StandardScaler()
X_train_bin_scaled = scaler_bin.fit_transform(X_train_bin)
X_test_bin_scaled = scaler_bin.transform(X_test_bin)

print(f"Train: {len(X_train_bin):,} samples (BUY/SELL only)")
print(f"Test: {len(X_test_bin):,} samples")
print(f"BUY ratio (train): {y_train_bin.mean()*100:.1f}%")
print(f"BUY ratio (test): {y_test_bin.mean()*100:.1f}%")

# --- Train GPU Models (Binary Classification) ---
models_bin = {}

# 1. XGBoost (GPU)
models_bin['xgb1'] = xgb.XGBClassifier(
    n_estimators=500, max_depth=8, learning_rate=0.05, 
    subsample=0.8, colsample_bytree=0.8, random_state=42,
    tree_method='hist', device='cuda', verbosity=0
)

# 2. XGBoost (GPU) - Deeper
models_bin['xgb2'] = xgb.XGBClassifier(
    n_estimators=400, max_depth=12, learning_rate=0.03, 
    subsample=0.7, colsample_bytree=0.7, random_state=43,
    tree_method='hist', device='cuda', verbosity=0,
    reg_alpha=0.1, reg_lambda=1.0
)

# 3. LightGBM (GPU)
models_bin['lgb1'] = lgb.LGBMClassifier(
    n_estimators=500, max_depth=8, learning_rate=0.05,
    subsample=0.8, colsample_bytree=0.8, random_state=42, 
    verbose=-1, device='gpu'
)

# 4. LightGBM (GPU) - Different config
models_bin['lgb2'] = lgb.LGBMClassifier(
    n_estimators=700, max_depth=6, learning_rate=0.03,
    subsample=0.75, colsample_bytree=0.75, random_state=44, 
    verbose=-1, device='gpu', num_leaves=63
)

# 5. CatBoost (GPU)
models_bin['cat'] = CatBoostClassifier(
    iterations=500, depth=8, learning_rate=0.05,
    random_seed=42, task_type='GPU', devices='0',
    verbose=False
)

# Train
predictions_bin = {}
probabilities_bin = {}

print("\n🚀 Training Binary Models (GPU)...")
for name, model in models_bin.items():
    print(f"  Training {name.upper()}...", end=" ")
    model.fit(X_train_bin_scaled, y_train_bin)
    predictions_bin[name] = model.predict(X_test_bin_scaled)
    probabilities_bin[name] = model.predict_proba(X_test_bin_scaled)
    acc = accuracy_score(y_test_bin, predictions_bin[name])
    print(f"✓ Accuracy: {acc*100:.2f}%")

🔄 V6 BINARY: BUY vs SELL Only (HOLD хассан)
Train: 393,219 samples (BUY/SELL only)
Test: 80,296 samples
BUY ratio (train): 49.3%
BUY ratio (test): 52.2%

🚀 Training Binary Models (GPU)...
  Training XGB1... Train: 393,219 samples (BUY/SELL only)
Test: 80,296 samples
BUY ratio (train): 49.3%
BUY ratio (test): 52.2%

🚀 Training Binary Models (GPU)...
  Training XGB1... ✓ Accuracy: 49.75%
  Training XGB2... ✓ Accuracy: 49.75%
  Training XGB2... ✓ Accuracy: 49.66%
  Training LGB1... ✓ Accuracy: 49.66%
  Training LGB1... ✓ Accuracy: 49.14%
  Training LGB2... ✓ Accuracy: 49.14%
  Training LGB2... ✓ Accuracy: 49.50%
  Training CAT... ✓ Accuracy: 49.50%
  Training CAT... ✓ Accuracy: 50.40%
✓ Accuracy: 50.40%


In [8]:
# ================================================================
# V6 BINARY ENSEMBLE RESULTS
# ================================================================
print("="*70)
print("📊 V6 BINARY ENSEMBLE (BUY vs SELL)")
print("="*70)

# Weighted Ensemble
weights_bin = {'xgb1': 0.20, 'xgb2': 0.20, 'lgb1': 0.20, 'lgb2': 0.20, 'cat': 0.20}

final_proba_bin = np.zeros_like(probabilities_bin['xgb1'])
for name, w in weights_bin.items():
    final_proba_bin += w * probabilities_bin[name]

buy_prob = final_proba_bin[:, 1] * 100

# Model agreement bonus
all_agree_buy = np.all([predictions_bin[name] == 1 for name in models_bin.keys()], axis=0)
confidence_bin = buy_prob.copy()
confidence_bin[all_agree_buy] = np.minimum(confidence_bin[all_agree_buy] + 5, 100)

print(f"\nModel Agreement on BUY: {all_agree_buy.sum():,} signals")

# Results by Confidence
print("\n📊 BUY Signal Accuracy by Confidence:")
print("-"*60)
print(f"{'Confidence':>12} | {'Signals':>10} | {'Correct':>10} | {'Accuracy':>10}")
print("-"*60)

v6_results = {}
for conf in [50, 60, 70, 75, 80, 85, 90]:
    mask = confidence_bin >= conf
    if mask.sum() > 0:
        signals = mask.sum()
        correct = y_test_bin[mask].sum()
        acc = correct / signals * 100
        v6_results[conf] = {'signals': signals, 'correct': correct, 'accuracy': acc}
        print(f"{conf:>10}%+ | {signals:>10} | {correct:>10} | {acc:>9.1f}%")

# SELL (low confidence = SELL signal)
print("\n📊 SELL Signal Accuracy by Confidence:")
print("-"*60)
for conf in [50, 60, 70, 75, 80, 85, 90]:
    sell_conf = 100 - confidence_bin  # Invert for SELL
    mask = sell_conf >= conf
    if mask.sum() > 0:
        signals = mask.sum()
        correct = (y_test_bin[mask] == 0).sum()  # SELL = 0
        acc = correct / signals * 100
        print(f"{conf:>10}%+ | {signals:>10} | {correct:>10} | {acc:>9.1f}%")

📊 V6 BINARY ENSEMBLE (BUY vs SELL)

Model Agreement on BUY: 21,377 signals

📊 BUY Signal Accuracy by Confidence:
------------------------------------------------------------
  Confidence |    Signals |    Correct |   Accuracy
------------------------------------------------------------
        50%+ |      40122 |      20818 |      51.9%
        60%+ |      19827 |      10351 |      52.2%
        70%+ |       4542 |       2482 |      54.6%
        75%+ |       1436 |        789 |      54.9%
        80%+ |        423 |        241 |      57.0%
        85%+ |        114 |         71 |      62.3%
        90%+ |         16 |         10 |      62.5%

📊 SELL Signal Accuracy by Confidence:
------------------------------------------------------------
        50%+ |      40174 |      19097 |      47.5%
        60%+ |      11145 |       5248 |      47.1%
        70%+ |       1287 |        616 |      47.9%
        75%+ |        392 |        166 |      42.3%
        80%+ |        119 |         42 | 

In [14]:
# ================================================================
# V6 vs V5 COMPARISON
# ================================================================
print("="*70)
print("📊 V6 vs V5 COMPARISON")
print("="*70)

# Load V5 models for comparison
v5_dir = BASE_DIR / 'models' / 'signal_generator_v5'
try:
    v5_xgb = joblib.load(v5_dir / 'xgboost_v5.joblib')
    v5_lgb = joblib.load(v5_dir / 'lightgbm_v5.joblib')
    v5_rf = joblib.load(v5_dir / 'rf_v5.joblib')
    v5_scaler = joblib.load(v5_dir / 'scaler_v5.joblib')
    v5_feature_cols = joblib.load(v5_dir / 'feature_cols_v5.joblib')
    
    # Check missing features for V5
    missing = [c for c in v5_feature_cols if c not in test_binary.columns]
    for c in missing:
        test_binary[c] = 0
    
    X_test_v5 = test_binary[v5_feature_cols].values
    X_test_v5_scaled = v5_scaler.transform(X_test_v5)
    
    # V5 predictions
    v5_xgb_proba = v5_xgb.predict_proba(X_test_v5_scaled)
    v5_lgb_proba = v5_lgb.predict_proba(X_test_v5_scaled)
    v5_rf_proba = v5_rf.predict_proba(X_test_v5_scaled)
    
    v5_weights = [0.4, 0.35, 0.25]
    v5_proba = v5_weights[0] * v5_xgb_proba + v5_weights[1] * v5_lgb_proba + v5_weights[2] * v5_rf_proba
    v5_buy_prob = v5_proba[:, 1] * 100
    
    # Agreement bonus
    v5_pred = [(p[:, 1] > 0.5).astype(int) for p in [v5_xgb_proba, v5_lgb_proba, v5_rf_proba]]
    v5_agree = (v5_pred[0] == 1) & (v5_pred[1] == 1) & (v5_pred[2] == 1)
    v5_conf = v5_buy_prob.copy()
    v5_conf[v5_agree] = np.minimum(v5_conf[v5_agree] + 5, 100)
    
    print(f"\n{'Threshold':>12} | {'V5 Signals':>10} | {'V5 Acc':>8} | {'V6 Signals':>10} | {'V6 Acc':>8} | {'Winner':>8}")
    print("-"*75)
    
    for conf in [70, 75, 80, 85, 90]:
        # V5
        v5_mask = v5_conf >= conf
        v5_sig = v5_mask.sum()
        v5_acc = y_test_bin[v5_mask].mean() * 100 if v5_sig > 0 else 0
        
        # V6
        v6_mask = confidence_bin >= conf
        v6_sig = v6_mask.sum()
        v6_acc = y_test_bin[v6_mask].mean() * 100 if v6_sig > 0 else 0
        
        winner = "V6 ✓" if v6_acc > v5_acc else ("V5 ✓" if v5_acc > v6_acc else "TIE")
        print(f"{conf:>10}%+ | {v5_sig:>10} | {v5_acc:>7.1f}% | {v6_sig:>10} | {v6_acc:>7.1f}% | {winner:>8}")
        
except Exception as e:
    print(f"V5 comparison error: {e}")

📊 V6 vs V5 COMPARISON

   Threshold | V5 Signals |   V5 Acc | V6 Signals |   V6 Acc |   Winner
---------------------------------------------------------------------------
        70%+ |          0 |     0.0% |       4522 |    55.4% |     V6 ✓
        75%+ |          0 |     0.0% |       1415 |    56.6% |     V6 ✓
        80%+ |          0 |     0.0% |        410 |    58.3% |     V6 ✓
        85%+ |          0 |     0.0% |        106 |    64.2% |     V6 ✓
        90%+ |          0 |     0.0% |         21 |    76.2% |     V6 ✓

   Threshold | V5 Signals |   V5 Acc | V6 Signals |   V6 Acc |   Winner
---------------------------------------------------------------------------
        70%+ |          0 |     0.0% |       4522 |    55.4% |     V6 ✓
        75%+ |          0 |     0.0% |       1415 |    56.6% |     V6 ✓
        80%+ |          0 |     0.0% |        410 |    58.3% |     V6 ✓
        85%+ |          0 |     0.0% |        106 |    64.2% |     V6 ✓
        90%+ |          0 |     

In [15]:
# ================================================================
# SAVE V6 BINARY MODELS
# ================================================================
print("Saving V6 Binary Models...")

for name, model in models_bin.items():
    joblib.dump(model, MODEL_DIR / f'{name}_v6_bin.joblib')

joblib.dump(scaler_bin, MODEL_DIR / 'scaler_v6_bin.joblib')
joblib.dump(feature_cols, MODEL_DIR / 'feature_cols_v6.joblib')
joblib.dump(weights_bin, MODEL_DIR / 'weights_v6.joblib')

# Best config
best_v6 = {'threshold': 75, 'mode': 'BINARY_BUY_SELL', 'models': list(models_bin.keys())}
joblib.dump(best_v6, MODEL_DIR / 'config_v6.joblib')

print("✅ V6 Binary Models Saved!")
print(f"   Location: {MODEL_DIR}")

Saving V6 Binary Models...
✅ V6 Binary Models Saved!
   Location: c:\Users\Acer\Desktop\Forex-Signal-App\models\signal_generator_v6
✅ V6 Binary Models Saved!
   Location: c:\Users\Acer\Desktop\Forex-Signal-App\models\signal_generator_v6
